# Import package and data

In [0]:

# Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

# 1. click the link below and authorize it in google drive 
# 2. paste the link in tq he white box7lñ.ijkughtydesw

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint

import cv2
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import datetime
import re

Using TensorFlow backend.


# Merge image with label and sample

In [0]:
def get_file_name(time=datetime.datetime(2018, 11, 2, 3, 21, 2)):

# datetime.datetime.fromtimestamp(os.path.getctime(os.getcwd()))>datetime.datetime(2018, 10, 25, 3, 21, 2)
  # read img list csv, this file contains the label of images
  os.chdir("/content/drive/My Drive/Networkly/Emotion/Datasets/AffectNet_Database/Manually_Annotated_file_lists")
  print('read_file')
  data = pd.read_csv('training.csv')
  count_=pd.DataFrame(data.subDirectory_filePath.str.count('/'))
  data['main'],data['pic_name']=data.subDirectory_filePath.str.split('/').str

  os.chdir("/content/drive/My Drive/Networkly/Emotion/Datasets/AffectNet_Database/Manually_Annotated_compressed/Manually_Annotated_Images")

  # count all the images in this dir
  subdirs=os.listdir(os.getcwd())

  r = re.compile("[0-9]+$")
  subdirs = list(filter(r.match, subdirs)) # Read Not

  print('getting file name')
  cwd=os.getcwd()
  a=[]
  b=[]
  for subdir in tqdm(subdirs):
    path=os.path.join(os.getcwd(),subdir)
    for file in os.listdir(path):
      file_path=os.path.join(path,file)
      c_time=datetime.datetime.fromtimestamp(os.path.getctime(file_path))
      if c_time>time:
        a.append(file) #new file 
      else:
        b.append(file) #old file
  print('length of existing files is {}'.format(len(b)))
  print('length of updated files is {}'.format(len(a)))
  print('')
  a=pd.DataFrame(a)
  # a=pd.DataFrame(b)
  a.columns=['pic_name']

  # expression dsc
  data_dict={0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger',
  7: 'Contempt', 8: 'None', 9: 'Uncertain', 10: 'No-Face'}

  # a['name'],a[1]=a[0].str.split('.',columns = ['name','row']).str

  # join it with dsc data and count the frequency of each expressions
  merged=a.merge(data,how='left',on='pic_name')
  count_=merged.groupby('expression',as_index=False).count()

  # get value from dict
  count_['emotion_dsc']=count_[count_.expression.notna()].expression.apply(lambda x:data_dict.get(x))
  count_.head()
  print('expression count')
  print(count_[['emotion_dsc','pic_name']])
  return merged

In [0]:
# function 
def get_sample(major_case_smpl_size=1500):
  # collect all the rare cases
  rare_case=merged[(merged.main.notna())&(merged.expression.isin([7,5,4,3,6,2]))]

  # sample more frequent cases, each 20000
  frequent_case=merged[(merged.main.notna())&(merged.expression.isin([0,1]))]
  grouped = frequent_case.groupby('expression')
  frequent_case=grouped.apply(lambda x: x.sample(major_case_smpl_size,random_state=5))
  # combine 
  df_sample=pd.concat([rare_case,frequent_case],axis=0)
  print('sample size is {}'.format(df_sample.shape[0]))
  return df_sample

In [0]:
# function used to convert imgs
def read_img(call_back=0):
  
  # annotations.head()
  os.chdir("/content/drive/My Drive/Networkly/Emotion/Datasets/AffectNet_Database/Manually_Annotated_compressed/Manually_Annotated_Images")

  paths = df_sample["subDirectory_filePath"]
  labels = df_sample["expression"]
  image = []
  value = []
  
  print('start converting')
  for i in tqdm(range(len(labels))[call_back:]):
    if os.path.exists(str(paths[i])):
      tobegray = cv2.imread(paths[i])
      gray=cv2.cvtColor(tobegray, cv2.COLOR_BGR2GRAY)
      resized=cv2.resize(gray, (128,128))
      image.append(resized)
      value.append(labels[i])
      if i%8000==0 or i==len(labels)-1:
        np.save(str(i)+'_11_4_image.npy', image)
        np.save(str(i)+'_11_4_label.npy', value)
        print('file  {}  is saved'.format(str(i)+'_11_04_image.npy'))

In [0]:
# merged=get_file_name()
df_sample=get_sample(major_case_smpl_size=11500)

NameError: ignored

In [0]:
read_img(call_back=32000)

  0%|          | 0/36647 [00:00<?, ?it/s]

start converting


  0%|          | 1/36647 [00:00<3:20:29,  3.05it/s]

file  0_11_04_image.npy  is saved


 22%|██▏       | 8001/36647 [35:38<3:37:33,  2.19it/s]

file  8000_11_04_image.npy  is saved


 44%|████▎     | 16001/36647 [1:11:40<3:18:02,  1.74it/s]

file  16000_11_04_image.npy  is saved


 47%|████▋     | 17138/36647 [1:17:19<2:22:44,  2.28it/s]

In [0]:
import glob
glob.glob('*.npy')

In [0]:
os.chdir("/content/drive/My Drive/Networkly/Emotion/Datasets/AffectNet_Database/Manually_Annotated_compressed/Manually_Annotated_Images")
np.load('800010_27_label.npy').shape,np.load('48000_10_27_label.npy').shape,np.load('56884_10_27_label.npy').shape

a=np.load('800010_27_image.npy')
b=np.load('48000_10_27_image.npy')
c=np.load('56884_10_27_image.npy')

a1=np.load('800010_27_label.npy')
b1=np.load('48000_10_27_label.npy')
c1=np.load('56884_10_27_label.npy')

images=np.concatenate((a,b,c),axis=0)
labels=np.concatenate((a1,b1,c1),axis=0)

# np.save('11_1_final_images',images)
# np.save('11_1_final_labels',labels)


In [0]:
a=np.load('10_28_images_all.npy')
b=np.load('10_28_label_all.npy')

In [0]:
images=np.concatenate((a,images),axis=0)
labels=np.concatenate((b,labels),axis=0)

In [0]:
len(images)

In [0]:
np.save('11_1_final_images_combine_all_history',images)
np.save('11_1_final_labels_combine_all_history',labels)

In [0]:
# extract zar file

In [0]:
os.listdir(os.getcwd())

['Manually_Annotated_Images',
 'part2',
 'Manually_Annotated.part05.rar',
 'Manually_Annotated.part06.rar']

In [0]:

import os
os.chdir('/content/drive/My Drive/Networkly/Emotion/Datasets/AffectNet_Database/Manually_Annotated_compressed')
!apt-get install unrar
!unrar x Manually_Annotated.part06.rar


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 3 not upgraded.
Need to get 129 kB of archives.
After this operation, 322 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 unrar amd64 1:5.5.8-1 [129 kB]
Fetched 129 kB in 1s (168 kB/s)
Selecting previously unselected package unrar.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.5.8-1_amd64.deb ...
Unpacking unrar (1:5.5.8-1) ...
Setting up unrar (1:5.5.8-1) ...
update-alternatives: using /usr/bin/unrar-nonfree to provide /usr/bin/unrar (unrar) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/unrar.1.gz because associated file /usr/share/man/man1/unrar-nonfree.1.gz (of link group unrar) doesn't exist

UNRAR 5.50 freeware      Copyright (c) 1993-20

In [0]:
extract_file()

NameError: ignored